In [1]:
import pyscf
import pyscf.tools
import matplotlib as mpl
from matplotlib import pyplot as plt

from orbitalpartitioning import *

# Build molecule
molecule = """
 Fe 1.67785607 0.00052233 0.06475932
 O 0.00000000 0.00000000 -0.47099074
 Fe -1.67785607 -0.00052233 0.06475932
 Cl 1.87002704 -1.09796437 1.99091682
 Cl 2.93244917 -0.98210488 -1.47467288
 Cl 2.37160936 2.07954091 -0.50446591
 Cl -1.87002704 1.09796437 1.99091682
 Cl -2.93244917 0.98210488 -1.47467288
 Cl -2.37160936 -2.07954091 -0.50446591
"""

basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   False,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


# Do UHF
pymol.build()
mf = pyscf.scf.UHF(pymol)
mf.init_guess       = "minao"
mf.chkfile          = "scf.fchk"
mf.verbose          = 4
mf.max_cycle        = 100
mf                  = mf.newton()
mf.run()


# Do UDFT
mf = pyscf.dft.UKS(pymol)
mf.xc               = 'b3lyp'
mf.damp             = 0.5
mf.diis_start_cycle = 8
mf.chkfile          = "scf.fchk"
mf.init_guess       = "chkfile"
mf.verbose          = 4
mf.conv_tol         = 1e-8
mf.conv_tol_grad    = 1e-5
mf.max_cycle        = 100
mf                  = mf.newton()
mf.run()

/Users/nmayhall/opt/anaconda3/envs/pyscf3/lib/python3.10/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '






******** <class 'pyscf.soscf.newton_ah.SecondOrderUHF'> ********
method = SecondOrderUHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
number electrons alpha = 87  beta = 77
******** <class 'pyscf.scf.uhf.UHF'> Newton solver flags ********
SCF tol = 1e-09
conv_tol_grad = None
max. SCF cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_cycle_inner = 12
max_stepsize = 0.05
ah_start_tol = 1e+09
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_start_cycle = 1
ah_max_cycle = 40
ah_grad_trust_region = 2.5
kf_interval = 4
kf_trust_region = 5
canonicalization = True
max_memory 4000 MB (current use 0 MB)
Set conv_tol_grad to 3.16228e-05
  alpha nocc = 87  HOMO = 0.

In [2]:
#
# Flip the local spin of the first Fe atom ('0 Fe' in ao_labels)
#
idx_fe1 = pymol.search_ao_label('0 Fe')
dma, dmb = mf.make_rdm1()
dma_fe1 = dma[idx_fe1.reshape(-1,1),idx_fe1].copy()
dmb_fe1 = dmb[idx_fe1.reshape(-1,1),idx_fe1].copy()
dma[idx_fe1.reshape(-1,1),idx_fe1] = dmb_fe1
dmb[idx_fe1.reshape(-1,1),idx_fe1] = dma_fe1
dm = [dma, dmb]

#
# Change the spin and run the second pass for low-spin solution
#
pymol.spin = 0
mf = pyscf.dft.UKS(pymol)
mf.xc = 'b3lyp'
# Apply large level shift to avoid big oscillation at the beginning of SCF
# iteration.  This is not a must for BS-DFT.  Depending on the system, this
# step can be omitted.
mf.level_shift = 1.0
mf.conv_tol = 1e-4
mf.verbose = 4
mf.kernel(dm)

#
# Remove the level shift and converge the low-spin state in the final pass
#
mf.level_shift = 0
mf.conv_tol = 1e-9
mf.kernel(mf.make_rdm1())



******** <class 'pyscf.dft.uks.UKS'> ********
method = UKS
initial guess = minao
damping factor = 0
level_shift factor = 1.0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 0.0001
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/w4/q6n49yl10v1fxv8jnns_cmp40000gn/T/tmptt5go9a6
max_memory 4000 MB (current use 0 MB)
number electrons alpha = 82  beta = 82
XC library pyscf.dft.libxc version 6.1.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = b3lyp
    P. J. Stephens, F. J. Devlin, C. F. Chabalowski, and M. J. Frisch.,  J. Phys. Chem. 98, 11623 (1994)
small_rho_cutoff = 1e-07
Set gradient conv threshold to 0.01
init E= -5363.19875061278
  alpha nocc = 82  HOMO = 0.00964076516752392  LUMO = 0.924933133298017
  beta  nocc = 82  HOMO = 0.639983019747612  LUMO = 1.15404861536712
cycle= 1 E= -5362.82856176781  

-5363.1690371917475

# BS Natural Orbitals
Now form natural orbitals from the BS solution

In [17]:
Ca = mf.mo_coeff[0][:,mf.mo_occ[0]==1]
Cb = mf.mo_coeff[1][:,mf.mo_occ[1]==1]
Va = mf.mo_coeff[0][:,mf.mo_occ[0]==0]
Vb = mf.mo_coeff[1][:,mf.mo_occ[1]==0]
S = mf.get_ovlp()
Pa = Ca @ Ca.T
Pb = Cb @ Cb.T
Pspin = Pa - Pb
pyscf.tools.molden.from_mo(mf.mol, "scr/Ca.molden", Ca)
pyscf.tools.molden.from_mo(mf.mol, "scr/Cb.molden", Cb)
# pyscf.tools.cubegen.density(mf.mol, "scr/Pspin.cube", Pspin, resolution=None, margin=3.0);

X = scipy.linalg.sqrtm(S)
Xinv = np.linalg.inv(X)

# Sab = Ca.T @ S @ Cb
# U,s,V = np.linalg.svd(Sab)
Pspin_orth = X@(Pa+Pb)@X 
Pspin_orth = .5*(Pspin_orth + Pspin_orth.T)
s,U = np.linalg.eigh(Pspin_orth)


perm = abs(s-1).argsort()
s = s[perm]
U = U[:,perm]
pyscf.tools.molden.from_mo(mf.mol, "scr/natorbs.molden", U)

# gaps = []
for sidx,si in enumerate(s[0:100]):
    print(" %4i %12.8f %12.8f" %(sidx,si, 1-abs(si-1)))


nact = 12 
inds_act  = []
inds_doc  = []
inds_vir  = []

for sidx,si in enumerate(s):
    if sidx < nact:
        inds_act.append(sidx)
    else:
        if si > .5:
            inds_doc.append(sidx)
        else:
            inds_vir.append(sidx)  

Cdoc = Xinv@U[:,inds_doc]
Cact = Xinv@U[:,inds_act]
Cvir = Xinv@U[:,inds_vir]
print(Cdoc.shape)
print(Cact.shape)
print(Cvir.shape)


    0   0.99955335   0.99955335
    1   1.00044665   0.99955335
    2   0.99588175   0.99588175
    3   1.00411825   0.99588175
    4   0.73267573   0.73267573
    5   1.26732427   0.73267573
    6   0.73096995   0.73096995
    7   1.26903005   0.73096995
    8   0.66386736   0.66386736
    9   1.33613264   0.66386736
   10   0.00123833   0.00123833
   11   1.99876167   0.00123833
   12   0.00090601   0.00090601
   13   1.99909399   0.00090601
   14   0.00084776   0.00084776
   15   1.99915224   0.00084776
   16   0.00073173   0.00073173
   17   1.99926827   0.00073173
   18   0.00064891   0.00064891
   19   1.99935109   0.00064891
   20   0.00045948   0.00045948
   21   1.99954052   0.00045948
   22   0.00042410   0.00042410
   23   1.99957590   0.00042410
   24   0.00027352   0.00027352
   25   1.99972648   0.00027352
   26   0.00016626   0.00016626
   27   1.99983374   0.00016626
   28   0.00015422   0.00015422
   29   1.99984578   0.00015422
   30   0.00011327   0.00011327
   31   

# Enlarge by Exchange matrix

In [24]:
Ka,Kb = mf.get_j()
Kact_doc = Cact.T@(Ka+Kb)@Cdoc
Kact_vir = Cact.T@(Ka+Kb)@Cvir

U,s,Vh = np.linalg.svd(Kact_doc, full_matrices=True)
print(Vh.shape)
print(" singular values: <act|K|doc>")
for si in s:
    print(" %12.8f"%si)
Cdoc2 = Cdoc @ Vh
    
U,s,Vh = np.linalg.svd(Kact_vir, full_matrices=True)
print(" singular values: <act|K|vir> ")
for si in s:
    print(" %12.8f"%si)
Cvir2 = Cvir @ Vh


# Cact_new = np.hstack((Cact,Cdoc2[:,0:nact], Cvir2[:,0:nact]))
# Cdoc_new = Cdoc2[:,nact:-1]
# Cvir_new = Cvir2[:,nact:-1]
Cact_new = np.hstack((Cact,Cdoc2[:,0:nact]))
Cdoc_new = Cdoc2[:,nact:-1]
Cvir_new = Cvir2
print(" ndoc: ", Cdoc_new.shape)
print(" nact: ", Cact_new.shape)
print(" nvir: ", Cvir_new.shape)

print(np.linalg.norm(Cdoc_new.T @ S @ Cact_new))
print(np.linalg.norm(Cvir_new.T @ S @ Cact_new))
print(np.trace(Cdoc_new.T @ S @ Cact_new))
print(np.trace(Cvir_new.T @ S @ Cact_new))
pyscf.tools.molden.from_mo(mf.mol, "scr/Cact.molden", Cact_new)
pyscf.tools.molden.from_mo(mf.mol, "scr/Cdoc.molden", Cdoc_new)

Cact_new = pyscf.lo.PM(pymol).kernel(Cact_new)
pyscf.tools.molden.from_mo(mf.mol, "scr/Cact_new.molden", Cact_new)

(77, 77)
 singular values: <act|K|doc>
   7.64168853
   7.58795013
   6.56283643
   6.55351657
   6.46607210
   5.52960527
   5.51078602
   4.79914146
   4.52302235
   4.36796748
 singular values: <act|K|vir> 
   8.39061919
   8.13794173
   8.12562375
   8.06521838
   8.04570668
   7.98956343
   7.90879487
   7.50670862
   7.30985052
   7.20623953
 ndoc:  (184, 66)
 nact:  (184, 20)
 nvir:  (184, 97)
4.2656135509665676e-14
4.549616810411327e-14
3.0677590287361487e-15
2.177169893292946e-15


# Localize active space

In [ ]:
from pyscf import lo, gto

Cact = pyscf.lo.PM(pymol).kernel(Cact)
Cdoc = pyscf.lo.PM(pymol).kernel(Cdoc)
pyscf.tools.molden.from_mo(mf.mol, "scr/Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "scr/Cdoc.molden", Cdoc)

In [ ]:
s,U = np.linalg.eigh(X@(Pa+Pb)@X)
perm = abs(s).argsort()[::-1]
s = s[perm]
for sidx,si in enumerate(s):
    print(" %5i: %12.8f"%(sidx,si))


NameError: name 'np' is not defined